### 前言
- 在自然语言处理(NLP)领域中，情感分析是一项非常常见的任务。它的目标是判断文本的情感倾向，例如在社交媒体上的评论、产品评价、电影评论等数据中，识别文本是正面的、负面的，还是中性的。与传统的二分类情感分析不同，许多应用场景下需要将情感分为更多类别，例如正面、负面和中性，这就是所谓的多分类情感分析。

- 本次分享将带你一步步使用 BERT(Bidirectional Encoder Representations fromTransformers)进行中文多分类情感分析。BERT是目前最强大的预训练语言模型之一，能够处理复杂的自然语言任务。通过BERT 的预训练模型，我们可以快速上手并进行模型微调，来完成情感分析任务。

- 本次我们将使用开源的 ChnSenticorp 数据集进行中文情感分析的多分类任务，包括数据清洗、模型训练、准确度评估以及模型导出等步骤。

### 中文情感分析的多分类任务简介
#### 情感分析的分类
- 情感分析旨在分析文本中的情感倾向。在传统的情感分析任务中，通常是将情感分类为“正面”和“负面”两类。多分类情感分析则需要分类更多的情感类别，比如“正面”、“中性” 三类，甚至“负面”、可以细化为不同的情感等级(如非常满意、满意、一般、差、非常差)。多分类任务的复杂性较高，因为情感的表达形式和种类多样，模型需要能够从文本的上下文中理解更细腻的情感差异。
### BERT 的优势
- BERT 模型通过预训练在大规模文本语料上学习到了丰富的语言表示，能够在许多 NLP 任务中达到顶尖水平。
- BERT 的双向特性使得它能够同时从句子的左右两边理解语义，这使它在情感分析任务中表现出色。

### 步骤概览
- 1.环境准备:安装所需的 Python 库和工具。
- 2.加载中文 BERT 预训练模型:使用 Huggingface 提供的 bert-base-chinese 模型。
- 3.加载开源数据集 ChnSenticorp:并进行数据清洗和预处理。
- 4.数据预处理:对文本进行分词、编码，并处理多分类标签
- 5.训练模型:对 BERT 进行微调，训练多分类情感分析模型。
- 6.评估模型性能:在测试集上评估模型的准确度。
- 7.导出模型:保存训练好的模型，供以后使用或部署。

In [ ]:
# sentiment_analysis.py

# 1. 环境准备
# 导入所需的库
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import jieba
import pandas as pd

# 2. 加载中文 BERT 预训练模型
model_name = 'bert-base-chinese'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)  # 2 表示二分类 (积极/消极)

# 3. 加载开源数据集 ChnSenticorp
# 假设 ChnSenticorp 数据集已经下载并解压到本地
# 请将 'path/to/chnsenticorp' 替换为实际的路径
try:
    dataset = load_dataset("csv", data_files={"train": "ChnSentiCorp_htl_all.csv"})
    # dataset = load_dataset("csv", data_files={"train": "path/to/chnsenticorp/train.csv", "test": "path/to/chnsenticorp/test.csv"}) # 如果有train和test文件
except FileNotFoundError:
    print("ChnSenticorp 数据集文件未找到，请确保文件路径正确。")
    exit()

# 数据清洗和预处理
def preprocess_function(examples):
    # 使用jieba分词
    tokenized_text = [" ".join(jieba.cut(text)) for text in examples["text"]]
    return tokenizer(tokenized_text, padding="max_length", truncation=True, max_length=128)

tokenized_datasets = dataset.map(preprocess_function, batched=True)

# 4. 数据预处理
# 将数据集转换为 PyTorch 张量
tokenized_datasets.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# 5. 训练模型
# 定义训练参数
batch_size = 32
learning_rate = 2e-5
epochs = 3

# 创建数据加载器
train_dataloader = torch.utils.data.DataLoader(tokenized_datasets["train"], shuffle=True, batch_size=batch_size)
# 检查是否有测试集，如果没有，则使用训练集的一部分作为测试集
if "test" in tokenized_datasets:
    eval_dataloader = torch.utils.data.DataLoader(tokenized_datasets["test"], batch_size=batch_size)
else:
    train_dataset, eval_dataset = torch.utils.data.random_split(tokenized_datasets["train"], [0.8, 0.2])
    eval_dataloader = torch.utils.data.DataLoader(eval_dataset, batch_size=batch_size)
    train_dataloader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=batch_size)

# 优化器
optimizer = AdamW(model.parameters(), lr=learning_rate)

# 训练循环
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

for epoch in range(epochs):
    model.train()
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(f"Epoch {epoch+1}/{epochs} 训练完成")

# 6. 评估模型性能
model.eval()
all_predictions = []
all_labels = []
with torch.no_grad():
    for batch in eval_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        all_predictions.extend(predictions.cpu().tolist())
        all_labels.extend(batch["label"].cpu().tolist())

accuracy = accuracy_score(all_labels, all_predictions)
print(f"准确率: {accuracy}")
print(classification_report(all_labels, all_predictions))

# 7. 导出模型
# 保存模型
model.save_pretrained("./sentiment_model")
tokenizer.save_pretrained("./sentiment_model")

print("模型已保存到 ./sentiment_model 目录")